This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***



 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [1]:
DATA_PATH = './obesity_data/'

**Classical Machine Learning - TF-IDF - All Features**

![CML TFIDF All](images\cml-tfidf-all.gif)

**Classical Machine Learning - TF-IDF - ExtraTreesClassifier Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-extra.gif)

**Classical Machine Learning - TF-IDF - InfoGain Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-infogain.gif)

**Classical Machine Learning - TF-IDF - SelectKBest Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-selectkbest.gif)

**Classical Machine Learning - Word Embeddings - No Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swno.gif)

**Classical Machine Learning - Word Embeddings - Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swyes.gif)

In [8]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime
from datetime import timedelta
from tqdm import tqdm
import torchtext

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, svm, naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
# define data path
DATA_PATH = './obesity_data/'
RESULTS_PATH = './results/'
MODELS_PATH = './models/'

all_docs_df = pd.read_pickle(DATA_PATH + '/alldocs_df.pkl')
all_docs_df_ns = pd.read_pickle(DATA_PATH + '/alldocs_df_ns.pkl')
all_annot_df = pd.read_pickle(DATA_PATH + '/alannot_df.pkl')

#corpus = pd.read_pickle(DATA_PATH + '/corpus.pkl')
disease_list = test_df['disease'].unique().tolist()

vectorizer = TfidfVectorizer(max_features = 600) #stop_words = cachedStopWords, max_features = 600

In [33]:
all_df = pd.merge(all_docs_df,all_annot_df, on='id')
all_df_ns = pd.merge(all_docs_df_ns,all_annot_df, on='id')

In [19]:
y = all_df['judgment']

In [39]:
all_df

,id,text,no_punc_text,no_numerics_text,lower_text,tokenized_text,tok_lem_text,disease,judgment
0,1,490646815 | WMC | 31530471 | | 9629480 | 11/23...,490646815 WMC 31530471 9629480 11232006 1...,WMC AM ANEMIA Signed DIS Admissi...,wmc am anemia signed dis admission date report...,"[wmc, am, anemia, signed, dis, admission, date...","[wmc, am, anemia, signed, dis, admission, date...",Asthma,False
1,1,490646815 | WMC | 31530471 | | 9629480 | 11/23...,490646815 WMC 31530471 9629480 11232006 1...,WMC AM ANEMIA Signed DIS Admissi...,wmc am anemia signed dis admission date report...,"[wmc, am, anemia, signed, dis, admission, date...","[wmc, am, anemia, signed, dis, admission, date...",CHF,True
2,1,490646815 | WMC | 31530471 | | 9629480 | 11/23...,490646815 WMC 31530471 9629480 11232006 1...,WMC AM ANEMIA Signed DIS Admissi...,wmc am anemia signed dis admission date report...,"[wmc, am, anemia, signed, dis, admission, date...","[wmc, am, anemia, signed, dis, admission, date...",Depression,False
3,1,490646815 | WMC | 31530471 | | 9629480 | 11/23...,490646815 WMC 31530471 9629480 11232006 1...,WMC AM ANEMIA Signed DIS Admissi...,wmc am anemia signed dis admission date report...,"[wmc, am, anemia, signed, dis, admission, date...","[wmc, am, anemia, signed, dis, admission, date...",Diabetes,True
4,1,490646815 | WMC | 31530471 | | 9629480 | 11/23...,490646815 WMC 31530471 9629480 11232006 1...,WMC AM ANEMIA Signed DIS Admissi...,wmc am anemia signed dis admission date report...,"[wmc, am, anemia, signed, dis, admission, date...","[wmc, am, anemia, signed, dis, admission, date...",Gallstones,False
...,...,...,...,...,...,...,...,...,...
16320,1247,216095265 | CCVGH | 79004464 | | 818191 | 10/1...,216095265 CCVGH 79004464 818191 10101997 ...,CCVGH AM CONGESTIVE HEART FAILURE ...,ccvgh am congestive heart failure rule out myo...,"[ccvgh, am, congestive, heart, failure, rule, ...","[ccvgh, am, congestive, heart, failure, rule, ...",OA,False
16321,1247,216095265 | CCVGH | 79004464 | | 818191 | 10/1...,216095265 CCVGH 79004464 818191 10101997 ...,CCVGH AM CONGESTIVE HEART FAILURE ...,ccvgh am congestive heart failure rule out myo...,"[ccvgh, am, congestive, heart, failure, rule, ...","[ccvgh, am, congestive, heart, failure, rule, ...",Obesity,True
16322,1247,216095265 | CCVGH | 79004464 | | 818191 | 10/1...,216095265 CCVGH 79004464 818191 10101997 ...,CCVGH AM CONGESTIVE HEART FAILURE ...,ccvgh am congestive heart failure rule out myo...,"[ccvgh, am, congestive, heart, failure, rule, ...","[ccvgh, am, congestive, heart, failure, rule, ...",OSA,False
16323,1247,216095265 | CCVGH | 79004464 | | 818191 | 10/1...,216095265 CCVGH 79004464 818191 10101997 ...,CCVGH AM CONGESTIVE HEART FAILURE ...,ccvgh am congestive heart failure rule out myo...,"[ccvgh, am, congestive, heart, failure, rule, ...","[ccvgh, am, congestive, heart, failure, rule, ...",PVD,False


In [20]:
X_train, X_test, y_train, y_test = train_test_split(all_df, y, test_size=0.25, shuffle=True)
X_train_ns, X_test_ns, y_train_ns, y_test_ns = train_test_split(all_df_ns, y, test_size=0.25, shuffle=True)

In [21]:
#y_train = all_df['judgement']

In [34]:
Encoder = LabelEncoder()
Train_Y  = Encoder.fit_transform(y_train)
Test_Y  = Encoder.fit_transform(y_test)

In [41]:
Tfidf_vect = TfidfVectorizer(max_features=600)
Train_X_Tfidf = vectorizer.fit_transform(X_train['lower_text'])
Test_X_Tfidf = vectorizer.fit_transform(X_test['lower_text'])

In [42]:
print(Train_X_Tfidf)
print(Tfidf_vect.vocabulary)

  (0, 141)	0.010501750360443594
  (0, 140)	0.004753545959583979
  (0, 155)	0.010685702549068274
  (0, 184)	0.010752777406464098
  (0, 522)	0.011272450812701633
  (0, 337)	0.011618677373828091
  (0, 583)	0.005913566910296181
  (0, 208)	0.008444341061781357
  (0, 108)	0.00634211772057659
  (0, 495)	0.0057969581637492195
  (0, 150)	0.006410488352462493
  (0, 213)	0.011640245400886104
  (0, 505)	0.029813564836250833
  (0, 550)	0.025300044198250066
  (0, 286)	0.013753253950078011
  (0, 342)	0.01167027197353661
  (0, 461)	0.020740474154472555
  (0, 484)	0.021220382967324144
  (0, 105)	0.009631205500512045
  (0, 531)	0.007662603994428354
  (0, 281)	0.014362456660890425
  (0, 156)	0.011199169652638289
  (0, 560)	0.01426013956862515
  (0, 254)	0.018843229087035993
  (0, 465)	0.010797100012253367
  :	:
  (12242, 29)	0.22099132574014543
  (12242, 258)	0.015852276408213923
  (12242, 28)	0.07533871523846838
  (12242, 584)	0.15870412752083518
  (12242, 593)	0.028291937123920422
  (12242, 276)	0.0521

# Support Vector Machine (SVM)

https://link.springer.com/article/10.1007/BF00994018

In [43]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)

# predict labels
predictions_SVM = SVM.predict(X_test)

# get the accuracy
print("Accuracy: ",accuracy_score(predictions_SVM, y_test)*100)

ValueError: could not convert string to float: "608074171 | CH | 62451179 | | 4737434 | 4/10/2006 12:00:00 AM | Discharge Summary | Unsigned | DIS | Admission Date: 7/14/2006 Report Status: Unsigned\n\nDischarge Date:\nATTENDING: MICHAELS , TYRELL MERLIN MD\nPRINCIPAL DIAGNOSES: Coronary artery disease , left ventricular\naneurysm.\nPROBLEM LIST:\n1. Congestive heart failure.\n2. Atrial fibrillation.\n3. Coronary artery disease.\nHISTORY OF PRESENT ILLNESS: Briefly , this is a 64-year-old\ngentleman with a history of an anterior myocardial infarction in\n1976 with resultant left ventricular aneurysm and congestive\nheart failure with an ejection fraction of approximately 20%. He\nhas had increasing chest pain with exertion over the last\nseveral years , which led to an evaluation for ischemia.\nCatheterization at the Ville Tionsgon County Hospital in Flint Ent Nyjer showed a 70% left\nmain lesion and a previously known 100% LAD , OM and RCA disease.\nThe patient had previously undergon

# k-Nearest Neighbours (kNN)

https://link.springer.com/article/10.1007/BF00153759

# Naive Bayes

https://arxiv.org/abs/1302.4964

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

# Random Forest

https://link.springer.com/article/10.1023/A:1010933404324

# Random Tree

https://onlinelibrary.wiley.com/doi/10.1002/rsa.3240050207

# J-48

http://server3.eca.ir/isi/forum/Programs%20for%20Machine%20Learning.pdf

# J-Rip

https://www.sciencedirect.com/science/article/pii/B9781558603776500232?via%3Dihub